# Get CrossRef Works

Use the DOIs from OpenAIRE to get publication metadata

## Preamble

In [ ]:
%run notebook_preamble.ipy

pd.set_option('max_columns', 99)

## CrossRef Works

In [ ]:
from crossref.restful import Etiquette

In [ ]:
etiquette = Etiquette(
    application_version='0.1',
    application_url='http://www.eurito.eu/',
    application_name='eu_funding_analytics',
    contact_email='george.richardson@nesta.org.uk',   
)

In [ ]:
def get_crossref_work(doi):
    works = Works(etiquette=etiquette)
    response = works.doi(doi)
    return response

In [ ]:
all_dois = publications_df['doi'][~pd.isnull(publications_df['doi'])].unique()

In [ ]:
test_dois = all_dois[:100]

In [ ]:
import json

In [ ]:
doi_chunks = list(chunks(all_dois, 1000))
doi_chunk_indices = list(range(len(doi_chunks)))

In [ ]:
start = 0
connections = 2 # API will rate limit occasionally with just 2 connections so needs babysitting

for i, dois in zip(doi_chunk_indices[start:], doi_chunks[start:]):
    out = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=connections) as executor:
        future_to_url = (executor.submit(get_crossref_work, doi) for doi in dois)
        for future in concurrent.futures.as_completed(future_to_url):
            data = future.result()
            out.append(data)
                
    with open(os.path.join(ext_data_path, 'crossref', 'works_{:04}.txt'.format(i)), 'w') as f:
        json.dump(out, f)